In [1]:
import os

import numpy as np

import os
import sys
sys.path.append('..')
#from layers import LSTMLayer, WaveNet, TimeDistributedDense, TemporalConvolution

In [2]:
import pandas as pd
from pandas import DataFrame

In [3]:
import tensorflow as tf
from tensorflow.keras import layers

In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!ls -a /content/drive/MyDrive/recsys_data/rnn_product_data

data.zip


In [ ]:
!mkdir data

In [ ]:
!unzip /content/drive/MyDrive/recsys_data/rnn_product_data/data.zip -d data

Archive:  /content/drive/MyDrive/recsys_data/rnn_product_data/data.zip
   creating: data/data/
  inflating: data/__MACOSX/._data    
  inflating: data/data/history_length.npy  
  inflating: data/__MACOSX/data/._history_length.npy  
  inflating: data/data/product_id.npy  
  inflating: data/__MACOSX/data/._product_id.npy  
  inflating: data/data/index_in_order_history.npy  
  inflating: data/__MACOSX/data/._index_in_order_history.npy  
  inflating: data/data/is_ordered_history.npy  
  inflating: data/__MACOSX/data/._is_ordered_history.npy  
  inflating: data/data/order_hour_history.npy  
  inflating: data/__MACOSX/data/._order_hour_history.npy  
  inflating: data/data/order_size_history.npy  
  inflating: data/__MACOSX/data/._order_size_history.npy  
  inflating: data/data/days_since_prior_order_history.npy  
  inflating: data/__MACOSX/data/._days_since_prior_order_history.npy  
  inflating: data/data/order_dow_history.npy  
  inflating: data/__MACOSX/data/._order_dow_history.npy  
  inf

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print(f'Running on a TPU w/{tpu.num_accelerators()["TPU"]} cores')
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Running on a TPU w/8 cores


In [5]:
class TFDataReader2:
    def __init__(self, data_dir):
        # Define feature columns and label columns
        self.feature_cols = [
            'user_id',
            'department_id',
            'is_ordered_history',
            'index_in_order_history',
            'order_dow_history',
            'order_hour_history',
            'days_since_prior_order_history',
            'order_size_history',
            'order_number_history',
            'num_products_from_department_history',
            'history_length',
        ]
        self.expand_cols = ['user_id', 'product_id', 'aisle_id', 'department_id',
            'history_length', 'product_name_length','label']

        # Load all numpy arrays
        self.data = {}
        for col in self.feature_cols:
            self.data[col] = np.load(os.path.join(data_dir, f'{col}.npy'), mmap_mode='r')
            #if col in self.expand_cols:
                #self.data[col] = self.data[col].reshape(-1,1)
        #rint(self.data.keys())
        # Create train/val split
        total_size = len(next(iter(self.data.values())))
        remainder = total_size % 512
        for col in self.feature_cols:
            self.data[col] = np.concatenate([self.data[col], np.zeros((512-remainder, *self.data[col].shape[1:]), dtype=self.data[col].dtype)], axis=0)
        train_size = int(0.9 * total_size)

        self.train_indices = np.arange(train_size)
        self.val_indices = np.arange(train_size, total_size)
        self.all_indices = np.arange(total_size + 512 - remainder)

    def _process_features(self, original_features, is_test):

        # Create new features dictionary with augmented features
        features = {
            # Copy original features
            **original_features,

            # Add augmented features
            'order_dow_history': tf.roll(original_features['order_dow_history'], -1, axis=0),
            'order_hour_history': tf.roll(original_features['order_hour_history'], -1, axis=0),
            'days_since_prior_order_history': tf.roll(original_features['days_since_prior_order_history'], -1, axis=0),
            'order_number_history': tf.roll(original_features['order_number_history'], -1, axis=0),
        }
        # Adjust history length for non-test data
        if not is_test:
            features['history_length'] = original_features['history_length'] - 1
        else:
            features['history_length'] = original_features['history_length']
        return features, {'next_is_ordered': tf.cast(tf.roll(original_features['is_ordered_history'], -1, axis=0), dtype=tf.float32)}

    def _create_dataset(self, indices, shuffle=True, is_test=False):
        # Create feature datasets
        features_dict = {col: tf.cast(self.data[col][indices], tf.int32) for col in self.feature_cols}

        dataset = tf.data.Dataset.from_tensor_slices(features_dict)
        # Apply processing before batching
        dataset = dataset.map(
            lambda x: self._process_features(x, is_test),
            num_parallel_calls=tf.data.AUTOTUNE
        )
        if shuffle:
            dataset = dataset.shuffle(buffer_size=10000)

        # Enable prefetching

        return dataset

    def get_train_dataset(self, batch_size):
        dataset = self._create_dataset(self.train_indices, shuffle=True)
        dataset = dataset.batch(batch_size, drop_remainder=True)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)

        #for element in dataset.take(1):
          #print(element[0])
        # Process features after batching
        #dataset = dataset.map(lambda x: self._process_features((x), is_test=False))
        return dataset

    def get_val_dataset(self, batch_size):
        dataset = self._create_dataset(self.val_indices, shuffle=True)
        dataset = dataset.batch(batch_size, drop_remainder=True)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)

        #dataset = dataset.map(lambda x: self._process_features((x), is_test=False))
        return dataset

    def get_test_dataset(self, batch_size):
        dataset = self._create_dataset(self.all_indices, shuffle=False, is_test=True)
        dataset = dataset.batch(batch_size, drop_remainder=False)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)

        #dataset = dataset.map(lambda x: self._process_features((x), is_test=True))
        return dataset

In [6]:
@tf.function
def sequence_log_loss(y, y_hat, sequence_lengths, max_sequence_length=100, eps=1e-7):
    # Ensure y is float32 for calculations
    #y = tf.cast(y, tf.float32)
    #print('casted', y.shape)
    # Clip predictions to avoid NaNs in log calculations
    y_hat = tf.clip_by_value(y_hat, eps, 1.0 - eps)
    #print('clipped', y_hat.shape)
    # Compute log losses
    log_losses = y * tf.math.log(y_hat) + (1.0 - y) * tf.math.log(1.0 - y_hat)
    #print("logged", log_losses.shape)
    # Create a sequence mask based on sequence lengths
    sequence_mask = tf.cast(tf.sequence_mask(sequence_lengths, maxlen=max_sequence_length), tf.float32)
    #print("mask created", sequence_mask.shape)
    # Apply the sequence mask to the log losses
    masked_log_losses = log_losses *sequence_mask
    #print("masked", masked_log_losses.shape)
    # Compute the average log loss
    avg_log_loss = -tf.reduce_sum(masked_log_losses) / tf.cast(tf.reduce_sum(sequence_lengths), tf.float32)
    #print("reduced")
    return avg_log_loss

In [7]:
class CustomModel(tf.keras.Model):
    def __init__(self, lstm_size, **kwargs):
        super(CustomModel, self).__init__(**kwargs)
        self.lstm_size = lstm_size
        self.lstm_layer = tf.keras.layers.GRU(lstm_size, return_sequences=True, dropout=0.0)
        self.dense1 = tf.keras.layers.Dense(50, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')
        self.user_embeddings = tf.keras.layers.Embedding(207000, lstm_size, name='user_embeddings')
        self.product_name_dense = tf.keras.layers.Dense(100, activation='relu')
    def call(self,inputs):
        user_id = inputs['user_id']
        department_id = inputs['department_id']
        self.history_length = inputs['history_length']

        is_ordered_history = inputs['is_ordered_history']
        index_in_order_history = inputs['index_in_order_history']
        order_dow_history = inputs['order_dow_history']
        order_hour_history = inputs['order_hour_history']
        days_since_prior_order_history = inputs['days_since_prior_order_history']
        order_size_history = inputs['order_size_history']
        order_number_history = inputs['order_number_history']
        num_products_from_department_history = inputs['num_products_from_department_history']

        

        #print(product_embeddings.shape, aisle_embeddings.shape, department_embeddings.shape, is_none_float.shape, product_names.shape)
        x_department = tf.one_hot(department_id, 30)
        x_department = tf.tile(tf.expand_dims(x_department, 1), (1, 100, 1))

        # User data
        user_embeddings = self.user_embeddings(user_id)
        x_user = tf.tile(tf.expand_dims(user_embeddings, 1), (1, 100, 1))

        # Sequence data
        is_ordered_history_onehot = tf.one_hot(is_ordered_history, 2)
        index_in_order_history_onehot = tf.one_hot(index_in_order_history, 20)
        order_dow_history_onehot = tf.one_hot(order_dow_history, 8)
        order_hour_history_onehot = tf.one_hot(order_hour_history, 25)
        days_since_prior_order_history_onehot = tf.one_hot(days_since_prior_order_history, 31)
        order_size_history_onehot = tf.one_hot(order_size_history, 60)
        order_number_history_onehot = tf.one_hot(order_number_history, 101)
        num_products_from_department_history_onehot = tf.one_hot(num_products_from_department_history, 50)


        index_in_order_history_scalar = tf.expand_dims(tf.cast(index_in_order_history, tf.float32) / 20.0, 2)
        order_dow_history_scalar = tf.expand_dims(tf.cast(order_dow_history, tf.float32) / 8.0, 2)
        order_hour_history_scalar = tf.expand_dims(tf.cast(order_hour_history, tf.float32) / 25.0, 2)
        days_since_prior_order_history_scalar = tf.expand_dims(tf.cast(days_since_prior_order_history, tf.float32) / 31.0, 2)
        order_size_history_scalar = tf.expand_dims(tf.cast(order_size_history, tf.float32) / 60.0, 2)
        order_number_history_scalar = tf.expand_dims(tf.cast(order_number_history, tf.float32) / 100.0, 2)
        num_products_from_department_history_scalar = tf.expand_dims(tf.cast(num_products_from_department_history, tf.float32) / 50.0, 2)


        x_history = tf.concat([
            is_ordered_history_onehot,
            index_in_order_history_onehot,
            order_dow_history_onehot,
            order_hour_history_onehot,
            days_since_prior_order_history_onehot,
            order_size_history_onehot,
            order_number_history_onehot,
            num_products_from_department_history_onehot,
            index_in_order_history_scalar,
            order_dow_history_scalar,
            order_hour_history_scalar,
            days_since_prior_order_history_scalar,
            order_size_history_scalar,
            order_number_history_scalar,
            num_products_from_department_history_scalar,
        ], axis=2)

        outputs = tf.concat([x_history, x_department, x_user], axis=2)
        h = self.lstm_layer(outputs)
        h = tf.concat([h,outputs], axis=-1)
        h = tf.keras.layers.TimeDistributed(self.dense1, name='hidden_states')(h)
        y_hat = tf.keras.layers.TimeDistributed(self.dense2)(h)
        y_hat = tf.squeeze(y_hat, axis=-1)
        
        final_temporal_idx = tf.stack([tf.range(tf.shape(self.history_length)[0]), self.history_length - 1], axis=1)
        final_states = tf.gather_nd(h, final_temporal_idx)
        final_predictions = tf.gather_nd(y_hat, final_temporal_idx)
        # final_states = tf.keras.layers.Lambda(lambda x: x, name='final_states')(final_states)
        # final_predictions = tf.keras.layers.Lambda(lambda x: x, name='final_predictions')(final_predictions)
        return {'next_is_ordered': y_hat, 'final_states': final_states, 'final_predictions': final_predictions}
    @tf.function
    def train_step(self, data):
        x, y = data
        history_length = x['history_length']

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            # Pass both prediction and history_length to loss
            loss = sequence_log_loss(y['next_is_ordered'], y_pred['next_is_ordered'], history_length, 100)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(y['next_is_ordered'], y_pred['next_is_ordered'])
        results = {m.name: m.result() for m in self.metrics}
        results['loss'] = loss
        return results
    @tf.function
    def test_step(self, data):
        x, y = data
        history_length = x['history_length']

        y_pred = self(x, training=False)
        # Pass both prediction and history_length to loss
        loss = sequence_log_loss(y['next_is_ordered'], y_pred['next_is_ordered'], history_length, 100)

        self.compiled_metrics.update_state(y['next_is_ordered'], y_pred['next_is_ordered'])

        # Return metrics and val_loss
        results = {m.name: m.result() for m in self.metrics}
        results['loss'] = loss
        return results




In [9]:
reader = TFDataReader2('data')
train_dataset = reader.get_train_dataset(256)
val_dataset = reader.get_val_dataset(512)

In [10]:
checkpoint_path = "/checkpoints/cp-{epoch:04d}.ckpt"


In [11]:
callbacks = [
    # Early stopping to prevent overfitting
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),

    # Model checkpoint to save best model
    tf.keras.callbacks.ModelCheckpoint(
        filepath='models/best_model.h5',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    ),

    # Learning rate reduction on plateau
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=1,
        min_lr=1e-6,
        verbose=1
    ),

    # CSV logger
    tf.keras.callbacks.CSVLogger(
        'training_log.csv',
        separator=',',
        append=False
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        save_freq='epoch',
        verbose=1
)

]


In [14]:

model = CustomModel(
        lstm_size=300,
    )
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(0.001),
    metrics=[
        tf.keras.metrics.Accuracy(name='accuracy'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.F1Score(name='f1')
    ]

)



In [15]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    steps_per_epoch=None,
    validation_steps=None,
    callbacks=callbacks,
    verbose=1,  # 0: silent, 1: progress bar, 2: one line per epoch
    shuffle=True,  # Shuffle training data
    max_queue_size=10,
)

Epoch 1/10


2024-12-20 09:21:46.953047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-12-20 09:21:47.909693: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-12-20 09:21:50.014930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2487/7849 [========>.....................] - ETA: 34:22 - accuracy: 0.0000e+00 - auc: 0.9037 - precision: 0.5934 - recall: 0.5363 - f1: 0.0333 - loss: 0.4966

KeyboardInterrupt: 

In [ ]:
model.save_weights(
            f'models/epoch_1.h5')

In [ ]:
eval_results = model.evaluate(val_dataset, verbose=1)
